In [11]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import pandas as pd
from google.colab.patches import cv2_imshow
import cv2
import os
from datetime import date,datetime
import csv
from deep_translator import GoogleTranslator

In [13]:
#@title Clases de la IA
CLASS_NAMES = {
        0: 'apple_pie', 1: 'baby_back_ribs', 2: 'baklava', 3: 'beef_carpaccio',
        4: 'beef_tartare', 5: 'beet_salad', 6: 'beignets', 7: 'bibimbap',
        8: 'bread_pudding', 9: 'breakfast_burrito', 10: 'bruschetta', 11: 'caesar_salad',
        12: 'cannoli', 13: 'caprese_salad', 14: 'carrot_cake', 15: 'ceviche',
        16: 'cheese_plate', 17: 'cheesecake', 18: 'chicken_curry', 19: 'chicken_quesadilla',
        20: 'chicken_wings', 21: 'chocolate_cake', 22: 'chocolate_mousse', 23: 'churros',
        24: 'clam_chowder', 25: 'club_sandwich', 26: 'crab_cakes', 27: 'creme_brulee',
        28: 'croque_madame', 29: 'cup_cakes', 30: 'deviled_eggs', 31: 'donuts',
        32: 'dumplings', 33: 'edamame', 34: 'eggs_benedict', 35: 'escargots',
        36: 'falafel', 37: 'filet_mignon', 38: 'fish_and_chips', 39: 'foie_gras',
        40: 'french_fries', 41: 'french_onion_soup', 42: 'french_toast', 43: 'fried_calamari',
        44: 'fried_rice', 45: 'frozen_yogurt', 46: 'garlic_bread', 47: 'gnocchi',
        48: 'greek_salad', 49: 'grilled_cheese_sandwich', 50: 'grilled_salmon',
        51: 'guacamole', 52: 'gyoza', 53: 'hamburger', 54: 'hot_and_sour_soup',
        55: 'hot_dog', 56: 'huevos_rancheros', 57: 'hummus', 58: 'ice_cream',
        59: 'lasagna', 60: 'lobster_bisque', 61: 'lobster_roll_sandwich',
        62: 'macaroni_and_cheese', 63: 'macarons', 64: 'miso_soup', 65: 'mussels',
        66: 'nachos', 67: 'omelette', 68: 'onion_rings', 69: 'oysters', 70: 'pad_thai',
        71: 'paella', 72: 'pancakes', 73: 'panna_cotta', 74: 'peking_duck', 75: 'pho',
        76: 'pizza', 77: 'pork_chop', 78: 'poutine', 79: 'prime_rib',
        80: 'pulled_pork_sandwich', 81: 'ramen', 82: 'ravioli', 83: 'red_velvet_cake',
        84: 'risotto', 85: 'samosa', 86: 'sashimi', 87: 'scallops', 88: 'seaweed_salad',
        89: 'shrimp_and_grits', 90: 'spaghetti_bolognese', 91: 'spaghetti_carbonara',
        92: 'spring_rolls', 93: 'steak', 94: 'strawberry_shortcake', 95: 'sushi',
        96: 'tacos', 97: 'takoyaki', 98: 'tiramisu', 99: 'tuna_tartare', 100: 'waffles'
    }

In [20]:
def modelo(nombre):
  IA = tf.keras.models.load_model(nombre)
  print("modelo cargado exitosamente")
  return IA

def prediccion(IA, imagen):
  img = imagen
  img_redimension = image.load_img(img, target_size=(150, 150))
  img_array = image.img_to_array(img_redimension)/255
  img_array = np.expand_dims(img_array, axis=0)

  prediccion = IA.predict(img_array)

  predict = zip(prediccion[0],list(range(0,101)))
  predict = list(predict)
  ordenados = sorted(predict, key=lambda predict : predict[0],reverse=True)
  print(ordenados)
  prediccion_numerica = np.argmax(prediccion, axis=1)[0]

  comida = CLASS_NAMES[prediccion_numerica]

  return comida


def lectura(): #tener los archivos en la misma ruta donde se ejecuta el codigo
  data_nutricion = pd.read_csv('nutrition2.csv')
  data_excel = pd.read_excel('tabla.xlsx')


  print("se leyeron ambos archivos correctamente")

  #procesado a data excel y a data nutricion

  data_nutri = data_nutricion[data_nutricion["weight"] == 100]


  filas = data_excel.iloc[1, :]
  data_excel.columns = filas
  data_excel.reset_index(inplace=True)
  data_excel.drop([0,1,2], inplace=True)
  data_excel.reset_index(inplace=True)
  data_excel.rename(columns={"Proteína (g)":"Proteina", "Calorías": "Calorias"}, inplace=True)


  data_excel.drop(["index", "level_0", "Codigo", "Humed. (g)", "Fósforo (mg)", "Potasio (mg)", "Grasas (g)", np.nan], inplace=True, axis=1)

  print("procesado del excel y la tabla nutricion exitosamente")

  return data_nutri, data_excel

def busqueda(predicion, data_nutri):
  informacion = data_nutri[data_nutri["label"] == predicion]
  informacion.reset_index(inplace=True)

  return informacion

def guardado(fecha_actual,informacion):

    #directorio_base=os.getcwd()+'\\registro_de_alimentacion.csv' USAR SOLO SI NO SE ESTA EJECUTANDO EN LA MISMA RUTA
    nombre_registro = "registro_de_alimentacion.csv"
    formato=datetime.strftime(fecha_actual,'%d/%m/%Y')

    informacion.insert(0,'Fechas',[formato])

    if not os.path.exists(nombre_registro):

        with open('registro_de_alimentacion.csv', mode='a' ,newline='') as registro:
            archivo=csv.writer(registro)
            archivo.writerow(list(informacion))
            archivo.writerow(list(informacion.loc[0,list(informacion.columns)]))
    else:

        with open('registro_de_alimentacion.csv', mode='a' ,newline='') as registro:
            archivo=csv.writer(registro)
            archivo.writerow(list(informacion.loc[0,list(informacion.columns)]))

def traduccion(prediccion):
  traduccion = GoogleTranslator(source='auto', target='es').translate(prediccion.replace("_", " "))
  return traduccion

def sugerencias(informacion, data_excel):
    nombre = informacion["label"][0]
    nombre_traducido = traduccion(nombre)
    df = data_excel

    if informacion["sugars"][0] > 15:

        print(f"{nombre_traducido} tiene alto porcentaje de azucar, evitar su consumo excesivo")
    else:

        if informacion["calories"][0] <= 400 and informacion["protein"][0] >= 20:

            if informacion["carbohydrates"][0] > 45:
              print(f"{nombre_traducido} tiene buen porcentaje en proteinas y carbohidratos pero es bajo en calorias, aqui unas sugerencias con que complementarlo: ")
              recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] < 20)]

            else:
              print(f"{nombre_traducido} tiene buen porcentaje en proteinas pero es bajo en calorias y carbohidratos, aqui unas sugerencias con que complementarlo: ")
              recomendaciones = df[(df['Calorias'] > 300) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] > 100 and informacion["protein"][0] < 20:

            if informacion["carbohydrates"][0] > 45:
                print(f"{nombre_traducido} tiene un buen porcentaje de calorias y carbohidratos pero es bajo en proteinas, aqui unas sugerencias con que complementarlo: ")
                recomendaciones = df[(df['Proteina'] > 20) & (df['Calorias'] < 100)]
            else:
                print(f"{nombre_traducido} tiene un buen porcentaje de calorias pero es bajo en proteinas y carbohidratos, aqui unas sugerencias con que complementarlo: ")
                recomendaciones = df[(df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] < 300 and informacion["protein"][0] < 20:

            if informacion["carbohydrates"][0] > 45:
                print(f"{nombre_traducido} tiene buen porcentaje de carbohidratos pero es bajo en calorias y en proteinas, te recomendamos aumentar ambas con unos alimentos sugeridos: ")
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20)]
            else:
                print(f"{nombre_traducido} es bajo en calorias, en proteinas y en carbohidratos, te recomendamos aumentar ambas con unos alimentos sugeridos: ")
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]
        else:

            print(f"{nombre_traducido} es alto en calorias y en proteinas pero bajo en carbohidratos, aqui unas sugerencias para complementarlo: ")
            recomendaciones = df[(df['Carbohidratos'] > 25) & (df['Calorias'] < 300) & (df['Proteina'] < 20)]


        if not recomendaciones.empty:
          recomendaciones = recomendaciones.sample(n=min(3, len(recomendaciones)))
          #print(recomendaciones)
          print("Te recomendamos las siguientes comidas:")
          for _, row in recomendaciones.iterrows():
              nombre_recomendado = row['Alimento']
              print(f"- {nombre_recomendado} (Calorías: {row['Calorias']}, Proteínas: {row['Proteina']}, Carbohidratos: {row['Carbohidratos']})")





IA = modelo("apa.h5")

ruta = "77987.jpg" #colocar aqui el nombre de la imagen o la ruta

prediccion = prediccion(IA, ruta)

data_nutri, data_excel = lectura() #dataframes

informacion = busqueda(prediccion, data_nutri) #data frame alimento

sugerencias(informacion, data_excel) #recomendaciones

guardado(date.today(), informacion)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


modelo cargado exitosamente
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
[(0.66029775, 45), (0.1311844, 15), (0.042175893, 58), (0.031294852, 23), (0.022774851, 60), (0.020637188, 27), (0.012466932, 92), (0.011593915, 86), (0.010667, 73), (0.008502034, 100), (0.0072309435, 32), (0.006139246, 95), (0.005060622, 29), (0.004749234, 83), (0.0032001643, 22), (0.0028891203, 36), (0.0020967624, 85), (0.001510744, 21), (0.0014023386, 99), (0.0012919109, 82), (0.00126205, 14), (0.0010506067, 65), (0.0009226243, 0), (0.000917774, 94), (0.0008427945, 13), (0.00077905844, 62), (0.00069949124, 35), (0.00063273247, 63), (0.0005190524, 4), (0.00050489773, 69), (0.0004909811, 12), (0.00044668626, 31), (0.0004202869, 98), (0.00027972806, 30), (0.00023373314, 68), (0.00020612913, 59), (0.00020136993, 55), (0.00019999882, 66), (0.00019969654, 9), (0.00019557799, 43), (0.00016015218, 16), (0.0001367682, 2), (0.00012737868, 54), (0.00011427128, 93), (9.570714e-05, 11), (9.460384e-05, 51), (9.195625e-05, 10), (8.